In [4]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import pandas as pd
from time import sleep
from random import randint
import datetime

In [5]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [6]:
city = 'Pittsburgh'

# Create empty list to store scraped data
datatoget = []

for counter, i in enumerate(range(0,960,120)):   
    
    # Set url
    url = f'https://pittsburgh.craigslist.org/search/pittsburgh-pa/apa?s={i}'
    browser.visit(url)
    sleep(randint(5,7))

    # Parse the HTML
    html = browser.html
    soup = bs(html, 'html.parser')

    # Find and extract relevant data
    search = soup.find_all('div', class_='result-info')
    
    print('\n====================================================')
    print(f'This page ({counter+1}) displays a range of {i} to {i+120} entries')
    print('====================================================')   

    # Loop through data found and extract relevant information
    for (counter,link) in enumerate(search):


        # Scrape dates
        dates = link.find_all('time', class_='result-date')[0].text
        
        # Scrape titles
        titles = link.find_all('a', class_='result-title hdrlnk')[0].text

        # Scrape links
        links = link.find_all('a', class_='result-title hdrlnk', href=True)[0]['href']

        # Scrape prices
        try:
            prices = link.find_all('span', class_='result-price')[0].text
        
        except:
            prices = np.nan
            
        # Scrape neighborhoods and handle any errors if no neighborhood found
        try:
            neighborhoods = link.find_all('span', class_='result-hood')[0].text.strip()[1:-1]

        except:
            neighborhoods = np.nan


        # Visit each link seperately
        browser.visit(links)
        
        sleep(randint(2,5))  

        # After visiting link, extract bath and amenities data
        html = browser.html
        soup = bs(html, 'html.parser')
        
        try:
            baths = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split()[2][:-2]

        except:
            baths = np.nan
        
        # Error handeling in case there are no sqft displayed
        try:
            if soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][-3:] == 'ft2':
                sqft = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][:-3]
            else:
                sqft = np.nan
        except:
            sqft = np.nan

        # Error handeling in case there are no bedrooms displayed
        try:
            bedrooms = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split('/')[0].strip().lower()

        except:
            bedrooms = np.nan

        try:
            amenities = soup.find_all('p', class_='attrgroup')[1].text.split('\n')
            # List comprehension used to make sure no empty strings get added to the list
            all_amenities = [am for am in amenities if am != '']
        
        except:
              all_amenities = np.nan

        # Print statements to ensure that the correct data is scraped
        print(f'\nListing Number: {counter+1}')
        print(f'Square Footage: {sqft if sqft == sqft else "not available"}')
        print(f'Price: {prices if prices == prices else "not available"}')
        print(f'Title: {titles}')
        print(f'Date: {dates}')
        print(f'Number of Bedrooms: {bedrooms if bedrooms == bedrooms else "not available"}')
        print(f'Neighborhood: {neighborhoods if neighborhoods == neighborhoods else "not available"}')
        print(f'Number of Bathrooms: {baths if baths == baths else "not available"}')
        print(f'Link: {links}')
        print(f'Amenities: {all_amenities}')

        # Create dicitonary for our DataFrame
        data = {'date': dates,
                'title': titles,
                'link': links,
                'price': prices,
                'bedroom': bedrooms,
                'sqft': sqft,
                'neighborhood': neighborhoods,
                'bathroom': baths,
                'amenities': all_amenities}

        # Append the data
        datatoget.append(data)

print('\n================ Done ================')

# Quit automated browser instance
browser.quit()


This page (1) displays a range of 0 to 120 entries

Listing Number: 1
Square Footage: 375
Price: $825
Title: 24 Hour Emergency Maintenance, Hardwood Flooring, Newly Painted
Date: Jan 18
Number of Bedrooms: 1br
Neighborhood: 1144 Tennessee Ave - D10, Pittsburgh, PA
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/pittsburgh-24-hour-emergency/7575672675.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry on site', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 2
Square Footage: 860
Price: $1,350
Title: Free Parking, Pet Friendly (Pet Fees Apply), Online Payments accepted
Date: Jan 18
Number of Bedrooms: 2br
Neighborhood: 7 Dorchester Drive, Pittsburgh, PA
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/pittsburgh-free-parking-pet-friendly/7579314782.html
Amenities: ['apartment', 'laundry in bldg', 'off-street parking', 'rent period: monthly']

Listing Number: 3
Square Footage: 1035
Pr


Listing Number: 20
Square Footage: not available
Price: $1,300
Title: Lovely, Bright, 2BD 1BA Updated Apartment w/ Balcony
Date: Jan 18
Number of Bedrooms: 2br
Neighborhood: Brighton Heights
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/pittsburgh-lovely-bright-2bd-1ba/7579997041.html
Amenities: ['wednesday 2023-01-25']

Listing Number: 21
Square Footage: not available
Price: $1,200
Title: 2BR 1BA WASHER/DRYER IN UNIT
Date: Jan 18
Number of Bedrooms: 2br
Neighborhood: CRAFTON
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/pittsburgh-2br-1ba-washer-dryer-in-unit/7580003141.html
Amenities: ['apartment', 'w/d in unit', 'no smoking', 'street parking', 'rent period: monthly']

Listing Number: 22
Square Footage: 950
Price: $1,650
Title: 3B/2B Modern SS Home
Date: Jan 18
Number of Bedrooms: 3br
Neighborhood: 68 S. 16th Street, PA
Number of Bathrooms: 2
Link: https://pittsburgh.craigslist.org/apa/d/pittsburgh-3b-2b-modern-ss-home/7575642069.html
Am


Listing Number: 39
Square Footage: 1
Price: $825
Title: One Bedroom Mckeesport Apartment
Date: Jan 18
Number of Bedrooms: 1br
Neighborhood: Versailles Ave
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/mckeesport-one-bedroom-mckeesport/7579987915.html
Amenities: ['air conditioning', 'apartment', 'laundry in bldg', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 40
Square Footage: not available
Price: $825
Title: 1BR Apartment in North Hills on Perry Hwy W/ Parking and Heat Included
Date: Jan 18
Number of Bedrooms: 1br
Neighborhood: pittsburgh
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/pittsburgh-1br-apartment-in-north-hills/7579987034.html
Amenities: ['cats are OK - purrr', 'apartment', 'laundry on site', 'off-street parking', 'rent period: monthly']

Listing Number: 41
Square Footage: not available
Price: $919
Title: CALL NOW!! Move in ASAP!!🏃.🏃‍♀️
Date: Jan 18
Number of Bedrooms: 1br
Neighborhood: Bethel Pa


Listing Number: 58
Square Footage: not available
Price: $849
Title: $99 Deposit Special🔥Huge Savings💰
Date: Jan 18
Number of Bedrooms: 1br
Neighborhood: Bethel Park
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/bethel-park-99-deposit-specialhuge/7573924861.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'apartment', 'laundry in bldg', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 59
Square Footage: not available
Price: $849
Title: Don't Wait, ☎️ Today $99 Deposit Special
Date: Jan 18
Number of Bedrooms: 1br
Neighborhood: Bethel Park
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/bethel-park-dont-wait-today-99-deposit/7573146302.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'apartment', 'laundry in bldg', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 60
Square Footage: not available
Price: $799
Title: 💥Best Bargain in Bethel💥$99 Deposit 💥
Date: Jan 18
Num


Listing Number: 77
Square Footage: not available
Price: $1,149
Title: 🚌🚙Easy Commute to Downtown🚙 🚌
Date: Jan 18
Number of Bedrooms: 2br
Neighborhood: South Park
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/south-park-easy-commute-to-downtown/7573102373.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'apartment', 'laundry in bldg', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 78
Square Footage: not available
Price: $1,049
Title: Stunning 1 Bedroom 🌈 $99 Deposit
Date: Jan 18
Number of Bedrooms: 1br
Neighborhood: South Park
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/south-park-stunning-bedroom-99-deposit/7573888954.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'apartment', 'laundry in bldg', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 79
Square Footage: not available
Price: $819
Title: $99 Deposit Special🔥Huge Savings💰
Date: Jan 18
Number of Bedroo

...


Listing Number: 37
Square Footage: not available
Price: $899
Title: Spacious! 2BR in Crafton - On Bus Line - Mins to Shopping - BALCONY!
Date: Jan 16
Number of Bedrooms: 2br
Neighborhood: Crafton
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/pittsburgh-spacious-2br-in-crafton-on/7579120127.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'apartment', 'laundry in bldg', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 38
Square Footage: not available
Price: $900
Title: LOVELY REMODELED 1 BEDROOM AVAILABLE
Date: Jan 16
Number of Bedrooms: 1br
Neighborhood: 3245 BEECHWOOD BOULEVARD
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/lovely-remodeled-bedroom-available/7572588487.html
Amenities: ['apartment', 'laundry in bldg', 'off-street parking', 'rent period: monthly']

Listing Number: 39
Square Footage: not available
Price: $875
Title: ONE BEDROOM AVAILABLE
Date: Jan 16
Number of Bedrooms: 1br
Neighborhood:


Listing Number: 57
Square Footage: not available
Price: $1,896
Title: tHE ABBEYVILLE FLOOR PLAN AT MONTEBELLO, A 55+ GATED DEL WEBB
Date: Jan 16
Number of Bedrooms: 2br
Neighborhood: Pittsburgh
Number of Bathrooms: 2
Link: https://pittsburgh.craigslist.org/apa/d/homestead-the-abbeyville-floor-plan-at/7579090213.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'furnished', 'apartment', 'w/d in unit', 'attached garage', 'rent period: monthly']

Listing Number: 58
Square Footage: not available
Price: $849
Title: You're Here. You're Home. You're Happy. 😌
Date: Jan 16
Number of Bedrooms: 1br
Neighborhood: Castle Shannon
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/pittsburgh-youre-here-youre-home-youre/7569911732.html
Amenities: ['air conditioning', 'application fee details: $35 Application Fee Per Person 18+', 'cats are OK - purrr', 'apartment', 'laundry in bldg', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 59
Square F


Listing Number: 76
Square Footage: not available
Price: $950
Title: 😍 Spacious 2 Bedroom Apartment - DON'T WAIT ☎️
Date: Jan 16
Number of Bedrooms: 2br
Neighborhood: Bethel Park
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/south-park-spacious-bedroom-apartment/7578120144.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'apartment', 'laundry in bldg', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 77
Square Footage: not available
Price: $919
Title: How To Live Better At Berkshire Apartments
Date: Jan 16
Number of Bedrooms: 1br
Neighborhood: Bethel Park
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/bethel-park-how-to-live-better-at/7578210881.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'apartment', 'laundry in bldg', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 78
Square Footage: not available
Price: $919
Title: 👌 Spacious Modern Living
Date: Jan 16
Num


Listing Number: 95
Square Footage: not available
Price: $925
Title: Winter Specials...Call For Details👀😊❤
Date: Jan 16
Number of Bedrooms: 2br
Neighborhood: Moon Township
Number of Bathrooms: 1.5
Link: https://pittsburgh.craigslist.org/apa/d/coraopolis-winter-specialscall-for/7579049977.html
Amenities: ['cats are OK - purrr', 'apartment', 'laundry in bldg', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 96
Square Footage: not available
Price: $800
Title: In A Great Location...Spacious Apartment..Winter Specials
Date: Jan 16
Number of Bedrooms: 1br
Neighborhood: Coraopolis
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/coraopolis-in-great-locationspacious/7579046885.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'apartment', 'laundry in bldg', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 97
Square Footage: not available
Price: $800
Title: $99 Security Deposit....Heat Included...12th month f


Listing Number: 115
Square Footage: not available
Price: $1,050
Title: Brick house available in Regency Park neighborhood
Date: Jan 16
Number of Bedrooms: 3br
Neighborhood: 526 Millers Ln, Pittsburgh, PA
Number of Bathrooms: 2
Link: https://pittsburgh.craigslist.org/apa/d/pittsburgh-brick-house-available-in/7577712678.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'furnished', 'house', 'w/d in unit', 'attached garage', 'rent period: monthly']

Listing Number: 116
Square Footage: not available
Price: $600
Title: Just the Right Size for you!!😍😍😍
Date: Jan 16
Number of Bedrooms: 0br
Neighborhood: North Versailles
Number of Bathrooms: 1
Link: https://pittsburgh.craigslist.org/apa/d/wilmerding-just-the-right-size-for-you/7579036673.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'apartment', 'laundry in bldg', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 117
Square Footage: not available
Price: $1,200
Title: Ca

In [7]:
# creating DataFrame
df = pd.DataFrame(datatoget)
df

,date,title,link,price,bedroom,sqft,neighborhood,bathroom,amenities
0,Jan 18,"24 Hour Emergency Maintenance, Hardwood Floori...",https://pittsburgh.craigslist.org/apa/d/pittsb...,$825,1br,375,"1144 Tennessee Ave - D10, Pittsburgh, PA",1,"[cats are OK - purrr, dogs are OK - wooof, apa..."
1,Jan 18,"Free Parking, Pet Friendly (Pet Fees Apply), O...",https://pittsburgh.craigslist.org/apa/d/pittsb...,"$1,350",2br,860,"7 Dorchester Drive, Pittsburgh, PA",1,"[apartment, laundry in bldg, off-street parkin..."
2,Jan 18,"Yoga Studio, Fully Equipped Kitchen, Stainless...",https://pittsburgh.craigslist.org/apa/d/pittsb...,"$1,695",2br,1035,"7 Dorchester Drive, Pittsburgh, PA",1.5,"[apartment, laundry in bldg, off-street parkin..."
3,Jan 18,Renovated Efficiency on bus line walk to shops,https://pittsburgh.craigslist.org/apa/d/pittsb...,$750,1br,NaN,Carrick,1,"[air conditioning, apartment, laundry in bldg,..."
4,Jan 18,2B Apartment Walking Distance From Campus,https://pittsburgh.craigslist.org/apa/d/pittsb...,"$1,250",2br,570,"430 Atwood Street, Pittsburgh, PA",1,"[cats are OK - purrr, dogs are OK - wooof, apa..."
...,...,...,...,...,...,...,...,...,...
955,Jan 16,Just the Right Size for you!!😍😍😍,https://pittsburgh.craigslist.org/apa/d/wilmer...,$600,0br,NaN,North Versailles,1,"[air conditioning, cats are OK - purrr, apartm..."
956,Jan 16,Call Before this Rare Opportunity is Gone📞📞📞,https://pittsburgh.craigslist.org/apa/d/mckees...,"$1,200",3br,NaN,White Oak,1.5,"[air conditioning, cats are OK - purrr, apartm..."
957,Jan 16,Two bedroom apt in Bellevue,https://pittsburgh.craigslist.org/apa/d/pittsb...,$995,2br,1200,Bellevue,1,"[air conditioning, apartment, w/d in unit, no ..."
958,Jan 16,Call or Text Us to see this awesome 1 bedroom!,https://pittsburgh.craigslist.org/apa/d/pittsb...,$989,1br,NaN,Dormont,1,"[air conditioning, cats are OK - purrr, apartm..."


In [8]:
# Exporting data to csv
df.to_csv(f'{city}_data_raw.csv', index=False)